# Соревнования на Kaggle

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [3]:
# Читаем наши данные
df = pd.read_csv('train.csv', sep=',')

In [4]:
dt = pd.read_csv('test.csv', sep=',')

In [5]:
# Проверяем и смотрим на наши данные
df.head(10)

,target,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13
0,1,0.00,3.51,0.04,0.0,2.39,203.0,7.95,23.71,124.00,143,148,1,2
1,0,0.27,8.18,3.27,6.0,4.17,108.0,10.24,14.84,83.42,135,203,1,1
2,0,0.34,5.32,0.07,2.0,4.06,77.0,24.20,15.80,66.29,115,35,3,2
3,1,NaN,2.77,2.20,7.0,3.34,134.0,7.23,9.56,92.35,169,181,3,3
4,0,0.62,3.97,2.92,7.0,5.30,25.0,17.80,11.71,33.09,168,51,2,2
5,1,0.08,0.34,2.28,6.0,3.35,27.0,17.00,6.01,41.93,174,125,1,3
6,1,0.70,7.57,4.14,4.0,6.17,33.0,12.11,10.83,44.76,86,91,1,1
7,0,0.74,4.70,4.76,0.0,6.28,NaN,12.49,12.55,55.52,84,178,1,2
8,0,0.41,6.24,3.15,5.0,4.56,64.0,13.51,15.94,54.63,100,91,3,1
9,0,0.13,5.99,4.76,5.0,3.33,128.0,22.11,16.88,85.54,86,108,1,2


In [6]:
# Смотрим на размерность матрицы данных
df.shape

(1000, 14)

In [7]:
# Смотрим на число NaN-ов в таблице
NAs = pd.concat([df.isnull().sum()], axis=1)
NAs[NAs.sum(axis=1) > 0]

,0
feature1,19
feature3,18
feature4,22
feature5,17
feature6,107


In [8]:
# Заменяем пропуски на среднее значение
df.fillna(df.mean(), inplace=True)

In [9]:
df.head(10)

,target,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13
0,1,0.000000,3.51,0.04,0.0,2.39,203.00000,7.95,23.71,124.00,143,148,1,2
1,0,0.270000,8.18,3.27,6.0,4.17,108.00000,10.24,14.84,83.42,135,203,1,1
2,0,0.340000,5.32,0.07,2.0,4.06,77.00000,24.20,15.80,66.29,115,35,3,2
3,1,0.495341,2.77,2.20,7.0,3.34,134.00000,7.23,9.56,92.35,169,181,3,3
4,0,0.620000,3.97,2.92,7.0,5.30,25.00000,17.80,11.71,33.09,168,51,2,2
5,1,0.080000,0.34,2.28,6.0,3.35,27.00000,17.00,6.01,41.93,174,125,1,3
6,1,0.700000,7.57,4.14,4.0,6.17,33.00000,12.11,10.83,44.76,86,91,1,1
7,0,0.740000,4.70,4.76,0.0,6.28,111.12542,12.49,12.55,55.52,84,178,1,2
8,0,0.410000,6.24,3.15,5.0,4.56,64.00000,13.51,15.94,54.63,100,91,3,1
9,0,0.130000,5.99,4.76,5.0,3.33,128.00000,22.11,16.88,85.54,86,108,1,2


In [10]:
dt.fillna(dt.mean(), inplace=True)

In [11]:
dt.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,Id
0,0.42,8.27,0.35,1.000000,4.57,22.0,20.74,5.25,39.23,25,39,2,1,0
1,0.65,4.13,2.84,6.000000,5.71,149.0,15.86,9.32,102.48,84,150,2,1,1
2,0.44,1.95,1.08,3.000000,4.78,95.0,14.55,19.81,74.24,206,64,1,2,2
3,0.71,1.31,2.51,7.000000,5.58,158.0,23.06,8.44,103.27,209,162,2,2,3
4,0.38,5.66,3.85,3.581466,4.73,91.0,13.90,7.74,66.64,124,44,2,3,4


Так же рассматривался вариант, когда пропуски заменялись не на среднее значение, а на медиану. Скор от этого был немного ниже, поэтому остановился на среднем.

In [12]:
# Смотрим сбалансированы ли классы, они не сбалансированы 
df['target'].value_counts()

0    549
1    451
Name: target, dtype: int64

In [18]:
# Разделяю предикоторы и отклики
Y = df['target']
X = df.drop(columns=['target'])

In [19]:
# Разбиваем наши данные на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 1234)  

Будем делать XGBoost, с подбором параметров с помощью GridSearch

In [20]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [21]:
# Перебираем параметры для XGboost-a, перебирались различные параметры, и в итоге остановился на таком списке параметров
grid_param = {  
    'n_estimators': [100, 200, 300, 400, 500, 80, 90, 150],
    'max_features': ['sqrt', 5, 7, 9, 10],
    'max_depth': [3, 4, 5, 6, 7, 9 ,11, 13],
    'reg_alpha': [0, 1],
    'reg_lambda': [0, 1]
}

In [22]:
grid_search_res = GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, seed=1234, 
                               
       objective='binary:logistic',                  
       colsample_bylevel=1, colsample_bytree=0.8, subsample=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, 
       min_child_weight=1, missing=None, nthread=-1,
       reg_alpha=0, reg_lambda=1, 
       scale_pos_weight=1, silent=True), n_jobs=-1,
       param_grid=grid_param,
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [23]:
grid_search_res.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
             estimator=XGBClassifier(colsample_bytree=0.8, nthread=-1,
                                     seed=1234, silent=True, subsample=0.8),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7, 9, 11, 13],
                         'max_features': ['sqrt', 5, 7, 9, 10],
                         'n_estimators': [100, 200, 300, 400, 500, 80, 90, 150],
                         'reg_alpha': [0, 1], 'reg_lambda': [0, 1]},
             scoring='accuracy')

In [24]:
print("Best params on dev set:")
print(grid_search_res.best_params_)

Best params on dev set:
{'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 0}


In [25]:
# Cтрою предсказание модели
y_pred_train = grid_search_res.predict(x_train)
y_pred_test = grid_search_res.predict(x_test)

In [49]:
# Смотрим на матрицу ошибок для тестовых данных
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,0,1
0,174,15
1,10,131


In [48]:
# Смотрим на матрицу ошибок для тренировочных данных
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,0,1
0,360,0
1,0,310


In [53]:
print(metrics.classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93       184
           1       0.93      0.90      0.91       146

    accuracy                           0.92       330
   macro avg       0.92      0.92      0.92       330
weighted avg       0.92      0.92      0.92       330



In [31]:
# Строим предсказание для наших тестовых данных
y_pred = grid_search_res.predict(dt.drop(columns=['Id']))

In [32]:
y_pred

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,

# Делаем калибровку


Попробуем сделать еще калибровку и посмотрим, как сильно изменились наши результаты

In [33]:
from sklearn.calibration import CalibratedClassifierCV

In [34]:
model_sigmoid = CalibratedClassifierCV(grid_search_res, cv=2, method='sigmoid')

In [35]:
model_sigmoid.fit(x_train, y_train)

CalibratedClassifierCV(base_estimator=GridSearchCV(cv=5, error_score='raise',
                                                   estimator=XGBClassifier(colsample_bytree=0.8,
                                                                           nthread=-1,
                                                                           seed=1234,
                                                                           silent=True,
                                                                           subsample=0.8),
                                                   n_jobs=-1,
                                                   param_grid={'max_depth': [3,
                                                                             4,
                                                                             5,
                                                                             6,
                                                                             7,
           

In [36]:
# Делаем калибровку и смотрим на скор
y_pred_train2 = model_sigmoid.predict(x_train)
y_pred_test2 = model_sigmoid.predict(x_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       185
           1       0.89      0.87      0.88       145

    accuracy                           0.90       330
   macro avg       0.90      0.89      0.90       330
weighted avg       0.90      0.90      0.90       330



In [51]:
# Смотрим на матрицу ошибок 
conf_mat = metrics.confusion_matrix(y_test, y_pred_test2)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,0,1
0,170,19
1,15,126


Исходя из матрицы ошибок и итогового скора после калиброки, было принято решение ее не делать и ограничиться просто XGBoost - ом.

In [43]:
ds = pd.read_csv('sample.csv', sep = ',')

In [44]:
# Строим предсказание для наших тестовых данных
y_pred2 = model_sigmoid.predict(dt.drop(columns = ['Id']))

In [46]:
# Записываем наши результаты 
pd.DataFrame({"Id": ds["Id"], "target": y_pred}).to_csv("submission1.csv", index=None)

In [52]:
# pd.DataFrame({"Id": ds["Id"], "target": y_pred2}).to_csv("submission2.csv", index=None)